## **STYLEGAN2 setup** 

In [ ]:
%tensorflow_version 1.x
from numpy import random
!nvidia-smi
import os
from google.colab import drive
from pathlib import Path
content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

In [ ]:
if os.path.isdir("/content/drive/My Drive/style_gan2_ada"):
    %cd "/content/drive/My Drive/style_gan2_ada/stylegan2-ada"
else:
    %cd "/content/drive/My Drive/"
    !mkdir style_gan2_ada
    %cd style_gan2_ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

In [ ]:
%cd "/content/drive/My Drive/style_gan2_ada/stylegan2-ada"

In [ ]:
%cd "/content/drive/My Drive/style_gan2_ada/stylegan2-ada"
!git config --global user.name "AROY13-PRAETORIAN"
!git config --global user.email "abhigyan.roy2019@vitstudent.ac.in"
!git fetch origin
!git checkout origin/main -- train.py

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
! kaggle datasets download -d 'ikarus777/best-artworks-of-all-time'

In [ ]:
!mkdir -p ~/.kaggle

# **DATASET TO .tfrecords**

In [ ]:
zip_path = "/content/drive/MyDrive/style_gan2_ada/stylegan2-ada/best-artworks-of-all-time.zip"
!unzip {zip_path} -d /content/dataset/

In [ ]:
!mv /content/dataset/images/images/Vincent_van_Gogh/* /content/dataset_art

In [ ]:
import glob
tifCounter = len(glob.glob1('/content/dataset_art',"*"))
tifCounter

In [ ]:
|!mkdir /content/drive/MyDrive/style_gan2_ada/stylegan2-ada/art_resized/

In [ ]:
import cv2
import os

path = '/content/dataset_art/'


im_size = 512  

images = []

for file in os.listdir(path):
        img = cv2.imread(path + '/' + file)  
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        cv2.imwrite('/content/dataset/Abstract_gallery/art_resized/'+str(file)+'_resized.jpg', img)


In [ ]:
dataset_path = "/content/dataset/Abstract_gallery/art_resized/"
dataset_name = "our_art" 

!python dataset_tool.py create_from_images /content/custom_dataset/{dataset_name} {dataset_path}

# **TRAINING**

In [ ]:
import pickle

In [ ]:
dataset_path = "/content/drive/MyDrive/custom_dataset" 
dataset_name = "our_art" 

In [ ]:
dataset_name = "our_art"
snapshot_count = 2
mirrored = True
mirroredY = False
metric_list = None
augs = "bgc"
resume_from = "/content/drive/MyDrive/style_gan2_ada/results/00006-our_art-mirror-auto1-bgc-resumecustom/network-snapshot-000032.pkl"
!python train.py --outdir /content/drive/MyDrive/style_gan2_ada/results --snap={snapshot_count} --data=/content/drive/MyDrive/custom_dataset/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from} --augpipe="bgc"

# **IMAGE GENERATION FROM PRETRAINED MODEL**

In [ ]:
%pip install opensimplex
output_path = drive_path / 'MyDrive' / 'stylegan2_ada_output'
output_path_images = output_path / 'images'
stylegan2_main_path = drive_path / 'MyDrive' / 'style_gan2_ada' / 'stylegan2-ada'
if not output_path.is_dir():
    %mkdir "{output_path}"
output_path_images = output_path / 'images'
if not output_path_images.is_dir():
    %mkdir "{output_path_images}"

In [ ]:
output_path_images

# **GENERATE IMAGES**

In [ ]:
seed_init = random.randint(10000)
nbr_images = 100

generation_from = '/content/drive/MyDrive/style_gan2_ada/results/00000-our_art-mirror-auto1-bgc-resumecustom/network-snapshot-000112.pkl' # path to final trained model

!python "{stylegan2_main_path / 'generate.py'}" generate-images \
    --outdir="{output_path_images}" --trunc=0.7 \
    --seeds={seed_init}-{seed_init+nbr_images-1} --create-grid \
    --network={generation_from}

# **LATENT SPACE EXPLORATION**

In [ ]:
output_path

In [ ]:
walk_types = ['line', 'sphere', 'noiseloop', 'circularloop']
latent_walk_output = output_path
explored_network =  "/content/drive/MyDrive/style_gan2_ada/results/00000-our_art-mirror-auto1-bgc-resumecustom/network-snapshot-000112.pkl"  # path to final trained model

seeds = [random.randint(20000) for i in range(20)]
print(','.join(map(str, seeds)))
print("Base seeds:", seeds)
!python "{stylegan2_main_path / 'generate.py'}" generate-latent-walk --network="{explored_network}" \
    --outdir="{latent_walk_output}" --trunc=0.7 --walk-type="{walk_types[2]}" \
    --seeds={','.join(map(str, seeds))} --frames {len(seeds)*20} #generates 400 frames

# **ESRGAN UPSCALING**

In [ ]:
frames = "/content/drive/MyDrive/stylegan2_ada_output/frames"
%cd {frames}
print("Time                     GPU        Used      Total")
!nvidia-smi --query-gpu=timestamp,name,memory.used,memory.free --format=csv,noheader
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
!nvidia-smi --query-gpu=timestamp,name,memory.used,memory.free --format=csv,noheader

In [ ]:
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

In [ ]:
#@title
import os
from google.colab import files
import shutil
upload_folder = 'upload'
result_folder = 'results'
if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

In [ ]:
#@title
%cd {frames}
final_output = [i for i in glob.glob("*.png")]
list_of_genrated_frames=[]
output = frames
for i in final_output:
  abc = output+"/" + i 
  list_of_genrated_frames.append(abc)
print(list_of_genrated_frames)

In [ ]:
move_for_upscaling = frames+"/"+"Real-ESRGAN/upload"
for i in list_of_genrated_frames:
  shutil.move(i, move_for_upscaling)

In [ ]:
#@title
%cd Real-ESRGAN
!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input upload --netscale 4 --outscale 3.5 --half --face_enhance

# **Generate your video again**

In [ ]:
import cv2
import numpy as np
import glob
img_array = []
for filename in glob.glob('/content/drive/MyDrive/stylegan2_ada_output/frames/Real-ESRGAN/results/*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
name = 'upscaled_video2' 
name = name+'.avi'
out = cv2.VideoWriter(name,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()